## Testing Automatic Differentiation of Derivative Operators:
#### Gradient, Jacobian, Divergence, Laplacian, Curl

In [1]:
import torch
import DRLPDE_nn

In [2]:
### Initial Variables
num_points = 2
dim_in = 4
dim_out = 3

x = torch.randn(num_points, dim_in).requires_grad_(True)
y = torch.empty(num_points, dim_out)

In [3]:
### [y0, y1, y2] = F(x0, x1, x2, x3)
#
# y0 = (x0)^2 exp(x1) + x2
# y1 = cos(x1) + sin(x0) + x2
# y2 = x1 x2 x3

y[:,0] = x[:,0]**2 * torch.exp(x[:,1]) + x[:,2]*x[:,1] + x[:,3]
y[:,1] = torch.cos( x[:,1] ) + torch.sin( x[:,0] ) + torch.exp(x[:,2]) + x[:,3]
y[:,2] = x[:,0] * x[:,1] * x[:,2] + x[:,3]

In [4]:
J = DRLPDE_nn.jacobian(y,x)

In [5]:
A = DRLPDE_nn.advection(y,x,J)

In [6]:
L = DRLPDE_nn.laplace(y,x,J)

In [7]:
time_deriv = J[:,:,-1]

torch.Size([10, 3, 4])

torch.Size([10, 3])

torch.Size([10, 3])

torch.Size([10, 3])

10

torch.Size([10, 3, 4])

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])